# Calculating Weighted Mean 

UXarray supports weighting means based on geometries, such as face areas. This user-guide section will discuss how to calculate variable means them using unstructured grid geometries as weights. Below showcases two examples using weighted mean with grid face areas or edge lengths as weights. 

Currently, `weighted-mean` does not support nodes as weights. 

In [ ]:
import uxarray as ux
import xarray as xr
import cartopy.crs as ccrs

import warnings

warnings.filterwarnings("ignore")

## Data

Data used in this section is a 3-random-variable dataset on a quad hexagon mesh mapped to the nodes, edges, and faces.

In [ ]:
grid_path = "../../test/meshfiles/ugrid/quad-hexagon/grid.nc"
quad_hex_data_path_edge_centered = "../../test/meshfiles/ugrid/quad-hexagon/random-edge-data.nc"
quad_hex_data_path_face_centered = "../../test/meshfiles/ugrid/quad-hexagon/random-face-data.nc"

uxds_face = ux.open_mfdataset(grid_path, quad_hex_data_path_face_centered)
uxds_edge = ux.open_mfdataset(grid_path, quad_hex_data_path_edge_centered)

The figure below visualizes the dataset. 

In [ ]:
(
    uxds_face.uxgrid.plot(line_color="black")
    * uxds_edge
    .plot(
        cmap="inferno", size=150, marker="square", clabel=None, tools=["hover"]
    )
    .relabel("Edge Data")
    * uxds_face
    .plot(
        cmap="inferno", size=150, marker="triangle", clabel=None, tools=["hover"]
    )
    .relabel("Face Data")
).opts(legend_position="top_right")

## Weighted Mean based on Face Areas
Here we first look at the data values on each face and the faces' respsective areas. 

In [ ]:
uxds_face['random_data_face'].values

In [ ]:
uxds_face.uxgrid.face_areas.data

We can simply call `.weighted_mean()` on the UXDataArray to compute the weighted mean. The differences between the weighted mean and the regular mean is small since the area differences across the faces are small. 

In [ ]:
result = uxds_face['random_data_face'].weighted_mean()
result.values

In [ ]:
unweighted_result = uxds_face['random_data_face'].mean()
unweighted_result.values

## Weighted Mean Based on Edge Length

Here we show the similar steps but for edge-centered data and the edge lengths. 

In [ ]:
uxds_edge['random_data_edge'].values

In [ ]:
uxds_edge.uxgrid.edge_node_distances.data

The differences between weighted and unweighted mean is more drastic (~0.1 value difference) since the edge lengths have a larger variance. 

In [ ]:
result = uxds_edge['random_data_edge'].weighted_mean()
result.values

In [ ]:
unweighted_result = uxds_edge['random_data_edge'].mean()
unweighted_result.values

## Custom Weights

While custom weights are supported in `weighted-mean`, it is crucial that the weight inputs are mapped on the same unstructured grid as the data for accurate calculations. 

Data can be stored in various configurations in the unstructured grid. The figure below shows two possible configuration for face-centered data and edge-centered data can be stored for this particular grid.  




<img src="../_static/examples/weightedmean/edge_face_weightedmean.png" width="600" alt="Optional Alt Text">